In [ ]:
from rasa_nlu.model import Metadata, Interpreter

In [2]:
nlu_md = """
## intent:greet
- hey
- hello
- hi
- good morning
- good evening
- hey there

## intent:goodbye
- bye
- goodbye
- see you around
- see you later
"""
%store nlu_md > nlu.md

print("The data has been successfully saved inside the nlu.md file! You can move on to the next step!")

Writing 'nlu_md' (str) to file 'nlu.md'.
The data has been successfully saved inside the nlu.md file! You can move on to the next step!


In [3]:
nlu_config = """
language: "en"
pipeline: spacy_sklearn
"""
#w = open("nlu_config.yml", "w")
#w.write(nlu_config)
#w.close
%store nlu_config > nlu_config.yml

print("The configuration has been successfully stored inside the nlu_config.yml file. You can now move on to the next step!")

Writing 'nlu_config' (str) to file 'nlu_config.yml'.
The configuration has been successfully stored inside the nlu_config.yml file. You can now move on to the next step!


In [ ]:
!python -m rasa_nlu.train -c C:\Users\Alex\Projects\rasabot\nlu_config.yml --data C:\Users\Alex\Projects\rasabot\nlu.md -o models --fixed_model_name nlu --project current --verbose

print("The NLU model has been trained successfully! You can move to the next step!")

In [ ]:
from rasa_nlu.model import Metadata, Interpreter
import json

def pprint(o):
 # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

interpreter = Interpreter.load('./models/current/nlu')
pprint(interpreter.parse('great'))

In [4]:
stories_md = """

## story 1
* greet
  - memory_visit
  - slot{"id" : "1"}
  
## story 2
* greet
  - memory_visit
  - slot{"id" : "5"}
  
## story 3
* greet
  - memory_visit
  - slot{"id" : "19"}
  
## story 4
* greet
  - memory_visit
  - slot{"id" : "56"}

## story 5
* greet
  - memory_visit
  - slot{"id" : "2"}

## story 6
* greet
  - memory_visit
  - slot{"id" : "100"}

## story 7
* greet
  - memory_visit
  - slot{"id" : "74"}

## say goodbye
* goodbye
  - utter_goodbye
  
## dialogue 
* greet
  - memory_visit
* goodbye
  - utter_goodbye

"""
%store stories_md > stories.md

print("The training stories have been successfully saved inside the stories.md file. You can move on to the next step!")

Writing 'stories_md' (str) to file 'stories.md'.
The training stories have been successfully saved inside the stories.md file. You can move on to the next step!


In [5]:
domain_yml = """
intents:
  - greet
  - goodbye

slots:
  id:
    type: text

actions:
- memory_visit
- utter_goodbye

templates:
  utter_goodbye:
  - text: "Hope to see ya again buddie"
  - text: "Don't lose yourself in that lands"
"""
%store domain_yml > domain.yml

print("The domain has been successfully saved inside the domain.yml file. You can move on to the next step!")

Writing 'domain_yml' (str) to file 'domain.yml'.
The domain has been successfully saved inside the domain.yml file. You can move on to the next step!


In [ ]:
!python -m rasa_core.train -d C:\Users\Alex\Projects\rasabot\domain.yml -s C:\Users\Alex\Projects\rasabot\stories.md -o models/dialogue

In [ ]:
import IPython
from IPython.display import clear_output, HTML, display
from rasa_core.agent import Agent
from rasa_core.interpreter import RasaNLUInterpreter
import time

interpreter = RasaNLUInterpreter('models/current/nlu')
messages = ["Hi! you can chat in this window. Type 'stop' to end the conversation."]
agent = Agent.load('models/dialogue', interpreter=interpreter)

def chatlogs_html(messages):
    messages_html = ""
    for m in messages:
        if m.endswith('.jpg'):
             messages_html += "<img src={}, alt='Tiger pub'></img>".format(m)
        else:
             messages_html += "<p>{}</p>".format(m)
    chatbot_html = """<div class="chat-window" {}</div>""".format(messages_html)
    return chatbot_html


while True:
    clear_output()
    display(HTML(chatlogs_html(messages)))
    time.sleep(0.3)
    a = input()
    messages.append(a)
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for r in responses:
        key = 'image' if 'image' in r.keys() else 'text'
        messages.append(r.get(key))

In [ ]:
!python -m rasa_core.run -d C:\Users\Alex\Projects\models\dialogue -u C:\Users\Alex\Projects\models/current/nlu --endpoints C:\Users\Alex\Projects\endpoints.yml

In [ ]:
!python -m rasa_core_sdk.endpoint --actions actions